## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 17
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_17


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 24.4 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Kansas City Chiefs,16,2022,29.1,75.0,8.1,305.1,104.7,12.1,22.3,...,10.2,107.8,21.8125,11.5,12.8750,0.082278,2.5000,0.625,3.2,3.333333
1,Philadelphia Eagles,16,2022,28.4,68.8,8.3,243.6,101.3,12.5,20.5,...,10.3,121.2,29.8750,13.0,13.0625,0.074919,1.5625,0.875,1.5,3.125000


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[16]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_17,0.444632,0.333554,0.215195,0.329768,0.399351,0.344500,0.405011
4,San Francisco 49ers,2022,week_17,0.296040,0.249208,0.111003,0.264915,0.362863,0.256806,0.354511
3,Buffalo Bills,2022,week_17,0.293294,0.119747,0.071510,0.210106,0.357648,0.210461,0.349604


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_17,0.444632,0.333554,0.215195,0.329768,0.399351,0.344500,0.405011
4,San Francisco 49ers,2022,week_17,0.296040,0.249208,0.111003,0.264915,0.362863,0.256806,0.354511
3,Buffalo Bills,2022,week_17,0.293294,0.119747,0.071510,0.210106,0.357648,0.210461,0.349604
1,Philadelphia Eagles,2022,week_17,0.321726,0.254059,0.149511,0.228866,0.349919,0.260816,0.346395
5,Miami Dolphins,2022,week_17,0.160554,0.243447,0.129617,0.284569,0.364735,0.236585,0.339213
7,Dallas Cowboys,2022,week_17,0.102824,0.085872,0.069366,0.193872,0.337914,0.157970,0.308527
13,Cincinnati Bengals,2022,week_17,0.019125,0.082635,0.052303,0.188053,0.348158,0.138055,0.307029
2,Detroit Lions,2022,week_17,0.006408,0.119257,0.074239,0.210264,0.344356,0.150905,0.302113
25,New England Patriots,2022,week_17,0.039409,0.052585,0.068193,0.194916,0.337691,0.138559,0.300406
14,Seattle Seahawks,2022,week_17,0.006234,0.060377,0.057971,0.188059,0.342083,0.130945,0.300102
